# EDA

In [6]:
# import data & stuff

from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np

import torch
import torchvision
import torchvision.transforms as T
from torchvision.io import read_image

folder = torchvision.datasets.ImageFolder("./traindata")

In [7]:
print(folder)

Dataset ImageFolder
    Number of datapoints: 4500
    Root location: ./traindata
